# Langchain Router Tutorial

The LangChain framework offers a versatile component called the Router Chain, which enables dynamic selection of a pre-defined chain from a set of chains based on the given input.

In this tutorial we are going to explore how you can use Router Chains.



![langchain-router](https://github.com/langchain-ai/langchain/assets/30792411/5dd4d082-3431-43ce-883f-e2fec06d433f)


**The following actions are executed here**:

The user produces a text based input.

The router selects the most appropriate chain from five options:
- Python programmer
- Poet
- Wikipedia Expert
- Graphical artist
- Legal Expert

The large language model responds.

In [1]:
!pip install langchain openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.3 MB/s eta 0:00:00


## Importations

In [ ]:
!pip install -U langchain-openai

In [5]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import BedrockChat

from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

## Creating And Mapping Prompts

In [6]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

model = 'gpt-3.5-turbo-0613'
llm = ChatOpenAI(model=model, temperature=0)

In [7]:
developer_template = """You are a very smart Python programmer. \
You provide answers for algorithmic and computer problems in Python. \
You explain the code in a detailed manner. \

Here is a question:
{input}\
"""

poet_template = """You are a poet who replies to creative requests with poems in English. \
You provide answers which are poems in the style of Lord Byron or Shakespeare. \

Here is a question:
{input}\
"""

wiki_template = """You are a Wikipedia expert. \
You answer common knowledge questions based on Wikipedia knowledge. \
Your explanations are detailed and in plain English.

Here is a question:
{input}\
"""

image_creator_template = """You create a creator of images. \
You provide graphic representations of answers using SVG images.

Here is a question:
{input}\
"""

legal_expert_template = """You are a UK or US legal expert. \
You explain questions related to the UK or US legal systems in an accessible language \
with a good number of examples.

Here is a question:
{input}\
"""

prompt_infos = [
        {
            'name': 'python programmer',
            'description': 'Good for questions about coding and algorithms',
            'prompt_template': developer_template
        },
        {
            'name': 'poet',
            'description': 'Good for generating poems for creative questions',
            'prompt_template': poet_template
        },
        {
            'name': 'wikipedia expert',
            'description': 'Good for answering questions about general knowledge',
            'prompt_template': wiki_template
        },
        {
            'name': 'graphical artist',
            'description': 'Good for answering questions which require an image output',
            'prompt_template': image_creator_template
        },
        {
            'name': 'legal expert',
            'description': 'Good for answering questions which are related to UK or US law',
            'prompt_template': legal_expert_template
        }
]

## Mapping Destination Chains

In [8]:
# map destination chains
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt_template = prompt_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

# Creating LLMRouterChain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)


python programmer: Good for questions about coding and algorithms
poet: Good for generating poems for creative questions
wikipedia expert: Good for answering questions about general knowledge
graphical artist: Good for answering questions which require an image output
legal expert: Good for answering questions which are related to UK or US law


## Creating Router Chain

In [9]:
# creating the router prompt
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# creating the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Multiple Prompt Chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [10]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

## Test it all out

In [11]:
# python programming question
print(chain.run('Write a code snippet with python to build a simple chatbot with Langchain.'))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
python programmer: {'input': 'Write a code snippet with python to build a simple chatbot with Langchain.'}
> Finished chain.
To build a simple chatbot with Langchain, we can use the langchain-python library. Langchain is an open-source library that provides natural language processing capabilities for building chatbots. 

To get started, you'll need to install the langchain-python library. You can do this by running the following command in your terminal:

```
pip install langchain-python
```

Once you have the library installed, you can use the following code snippet to build a simple chatbot:

```python
from langchain import Langchain

# Create a Langchain instance
langchain = Langchain()

# Define a function to handle user input and generate a response
def get_response(user_input):
    response = langchain.generate_response(user_input)
    return response

# Main chat loop
while True:
    # Get user input
    user_input = input("User: ")

 

In [12]:
# Poem question
print(chain.run("Write a poem with the theme of early morning sunshine"))



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


poet: {'input': 'Write a poem with the theme of early morning sunshine'}
> Finished chain.
In the realm where darkness fades,
And dawn's first light begins to blaze,
A golden hue, a gentle kiss,
Awakens nature's sweet abyss.

The morning sun, with radiant grace,
Unveils the world, a sacred space,
Its rays caress the dew-kissed ground,
As whispers of a new day sound.

The sleepy earth, now bathed in light,
Shakes off the remnants of the night,
The flowers bloom, their petals unfurled,
Embracing warmth from the early world.

The birds take flight, their songs arise,
A symphony that fills the skies,
Their melodies, like whispers soft,
Float on the breeze, aloft, aloft.

The trees, adorned in vibrant green,
Stretch their branches, tall and serene,
As sunlight dances through the leaves,
A tapestry that nature weaves.

The morning sun, a beacon bright,
Dispels the shadows with its might,
It paints the world in hues divine,
A masterpiece, a grand design.

Oh, early morning sunshine fair,
With

In [13]:
# Wiki
print(chain.run("Please tell us about Elon Musk."))



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


wikipedia expert: {'input': 'Elon Musk'}
> Finished chain.
Elon Musk is a well-known entrepreneur and business magnate who was born on June 28, 1971, in Pretoria, South Africa. He is the CEO and lead designer of SpaceX, CEO and product architect of Tesla, Inc., CEO of Neuralink, and founder of The Boring Company. Musk is also involved in various other ventures and initiatives, including SolarCity (now part of Tesla), OpenAI, and the Musk Foundation.

Musk gained prominence for his role in co-founding PayPal, an online payment company, which was later acquired by eBay. After the sale, he went on to establish SpaceX in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has since become a major player in the aerospace industry, successfully launching numerous rockets and becoming the first privately-funded company to send a spacecraft to the International Space Station.

In addition to SpaceX, Musk is widely recognized for his involveme

In [14]:
# Image generating
print(chain.run("Create an artistic rendering in the style of Vincent Van Gogh's famous 'Sunflowers' series."))



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


graphical artist: {'input': "Create an artistic rendering in the style of Vincent Van Gogh's famous 'Sunflowers' series."}
> Finished chain.
I'm sorry, but as a text-based AI, I am unable to create or provide visual content such as SVG images. However, I can try to describe an artistic rendering in the style of Vincent Van Gogh's 'Sunflowers' series for you:

An artistic rendering in the style of Vincent Van Gogh's 'Sunflowers' series would typically feature a vibrant and expressive depiction of sunflowers. Van Gogh's style is characterized by bold brushstrokes, intense colors, and a sense of movement. To create a similar rendering, you could start with a canvas or paper in a square format.

Using a combination of yellows, oranges, and browns, apply thick and textured brushstrokes to represent the sunflowers' petals. Van Gogh often used short, swirling strokes to convey the texture and energy of the flowers. The center of each sunflower can be depicted with a darker shade of brown or b

In [15]:
# Legal Regulation
print(chain.run("What are the main laws regulating digital products like software and apps in the United States?"))




> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


legal expert: {'input': 'What are the main laws regulating digital products like software and apps in the United States?'}
> Finished chain.
In the United States, there are several key laws that regulate digital products such as software and apps. These laws aim to protect consumers, ensure fair competition, and address intellectual property rights. Let's explore some of the main laws:

1. Copyright Law: Copyright law protects original works of authorship, including software and apps. It grants exclusive rights to the creators, preventing others from copying, distributing, or modifying their work without permission. Copyright protection is automatic upon creation, but registering with the U.S. Copyright Office strengthens legal rights and provides additional benefits.

2. Digital Millennium Copyright Act (DMCA): The DMCA addresses copyright infringement in the digital realm. It criminalizes the circumvention of technological measures used to protect copyrighted works, such as software 